Temperature value --> how creative we want out model to be

0 -> Temperature it means model is very safe it is not taking any bets. 

1 -> It will take risk it might generate wrong output but it is very creative.

In [34]:
mytoken = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [36]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    huggingfacehub_api_token=mytoken,
    task="text2text-generation",   
    model_kwargs={
        "temperature": 0,
        "max_length": 64
    }
)


In [39]:
output=llm.predict("What is the capital city of India")
print(output)

AttributeError: 'InferenceClient' object has no attribute 'post'